In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Load the data into a DataFrame (assuming the data is stored in a CSV file)
data = pd.read_csv('/content/Solarrr.csv')

In [ ]:
data.columns

Index(['Date & Time', '\tLDR Reading', '\t\tAngle', '\t\tVoltage\t',
       '\tCurrent\t', '\tPower'],
      dtype='object')

In [ ]:
data = data.rename(columns={'\tLDR Reading': 'LDR Reading',
                            '\t\tAngle': 'Angle',
                            '\t\tVoltage\t': 'Voltage',
                            '\tCurrent\t': 'Current',
                            '\tPower': 'Power'})

In [ ]:
data

,Date & Time,LDR Reading,Angle,Voltage,Current,Power
0,[2023-07-04 20:13:13.997],601,15,1.42,2.52,3.57
1,[2023-07-04 20:13:14.096],648,24,1.83,2.54,4.65
2,[2023-07-04 20:13:14.200],626,20,1.96,2.54,4.96
3,[2023-07-04 20:13:14.300],621,19,2.22,2.56,5.70
4,[2023-07-04 20:13:14.405],631,21,2.61,2.61,6.81
...,...,...,...,...,...,...
1784,[2023-07-04 20:16:15.805],405,-19,0.86,2.51,2.15
1785,[2023-07-04 20:16:15.907],397,-21,0.83,2.51,2.08
1786,[2023-07-04 20:16:16.012],396,-21,0.86,2.52,2.15
1787,[2023-07-04 20:16:16.112],405,-19,0.88,2.51,2.21


In [ ]:
type(data['Date & Time'][0])

str

In [ ]:
# Define a function to separate date and time
import datetime
def separate_date_time(date_string):
    date_string = date_string.strip('[]').split('.')[0]
    datetime_obj = datetime.datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')
    date = datetime_obj.date()
    time = datetime_obj.time()
    return date, time

In [ ]:
# Apply the function to the date column and create separate date and time columns
data['Date'], data['Time'] = zip(*data['Date & Time'].apply(separate_date_time))

In [ ]:
# Print the updated DataFrame
data["Time"]


0       20:13:13
1       20:13:14
2       20:13:14
3       20:13:14
4       20:13:14
          ...   
1784    20:16:15
1785    20:16:15
1786    20:16:16
1787    20:16:16
1788    20:16:16
Name: Time, Length: 1789, dtype: object

In [ ]:
df= data.copy()

In [ ]:
df

,Date & Time,LDR Reading,Angle,Voltage,Current,Power,Date,Time
0,[2023-07-04 20:13:13.997],601,15,1.42,2.52,3.57,2023-07-04,20:13:13
1,[2023-07-04 20:13:14.096],648,24,1.83,2.54,4.65,2023-07-04,20:13:14
2,[2023-07-04 20:13:14.200],626,20,1.96,2.54,4.96,2023-07-04,20:13:14
3,[2023-07-04 20:13:14.300],621,19,2.22,2.56,5.70,2023-07-04,20:13:14
4,[2023-07-04 20:13:14.405],631,21,2.61,2.61,6.81,2023-07-04,20:13:14
...,...,...,...,...,...,...,...,...
1784,[2023-07-04 20:16:15.805],405,-19,0.86,2.51,2.15,2023-07-04,20:16:15
1785,[2023-07-04 20:16:15.907],397,-21,0.83,2.51,2.08,2023-07-04,20:16:15
1786,[2023-07-04 20:16:16.012],396,-21,0.86,2.52,2.15,2023-07-04,20:16:16
1787,[2023-07-04 20:16:16.112],405,-19,0.88,2.51,2.21,2023-07-04,20:16:16


In [ ]:
col_drop=["Date & Time","Date","Time"]

In [ ]:
columns_present = [col for col in col_drop if col in df.columns]
df= df.drop(columns=columns_present)

In [ ]:
df

,LDR Reading,Angle,Voltage,Current,Power
0,601,15,1.42,2.52,3.57
1,648,24,1.83,2.54,4.65
2,626,20,1.96,2.54,4.96
3,621,19,2.22,2.56,5.70
4,631,21,2.61,2.61,6.81
...,...,...,...,...,...
1784,405,-19,0.86,2.51,2.15
1785,397,-21,0.83,2.51,2.08
1786,396,-21,0.86,2.52,2.15
1787,405,-19,0.88,2.51,2.21


In [ ]:
df["Current"]

0       2.52
1       2.54
2       2.54
3       2.56
4       2.61
        ... 
1784    2.51
1785    2.51
1786    2.52
1787    2.51
1788    2.51
Name: Current, Length: 1789, dtype: float64

In [ ]:
# Prepare the independent variables (features)
X = data[['Current', 'Voltage', 'Angle', 'LDR Reading']]


In [ ]:
# Prepare the dependent variable (target)
y = data['Power']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
# Create a linear regression model
model = LinearRegression()


In [ ]:
# Fit the model using the data
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Make predictions on the test data
predictions = model.predict(X_test)

In [ ]:
# Evaluate the model using metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)


In [ ]:
# Print the evaluation metrics
print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2-0.01)

Mean Squared Error: 0.0003800236577455886
Mean Absolute Error: 0.01485655166161419
R-squared: 0.9898091532165987


In [ ]:
# Get the last date and time in the dataset
last_date = data['Date'].iloc[-1]
last_time = data['Time'].iloc[-1]

In [ ]:
from datetime import datetime, timedelta
# Combine the last date and time into a single datetime object
last_datetime = datetime.combine(last_date, datetime.strptime(str(last_time), '%H:%M:%S').time())

In [ ]:
# Get the date and time for tomorrow
tomorrow = last_datetime + timedelta(days=1)
tomorrow_date = tomorrow.date()
tomorrow_time = tomorrow.time()

In [ ]:
# Select the independent variables (features)
independent_variables = ['Current', 'Voltage', 'Angle', 'LDR Reading']

In [ ]:
# Prepare the input data for tomorrow's prediction
last_known_values = df[independent_variables].iloc[-1]
prediction_data = pd.DataFrame([last_known_values], columns=independent_variables)


In [ ]:
# Set the date and time for tomorrow in the prediction data
prediction_data['Date'] = tomorrow_date
prediction_data['Time'] = tomorrow_time


In [ ]:
# Make the power generation prediction for tomorrow
tomorrow_power = model.predict(prediction_data[independent_variables])

In [ ]:
# Print the predicted power for tomorrow
print('Predicted Power for Tomorrow:', tomorrow_power[0],"Kwh")

Predicted Power for Tomorrow: 2.1969179074569434 Kwh


In [ ]:
prediction_data

,Current,Voltage,Angle,LDR Reading,Date,Time
1788,2.51,0.88,-18.0,410.0,2023-07-05,20:16:16
